Tussenstap: nieuwe labels aanmaken

In [ ]:
gerwel = []
df_unmatched['lowertag'] = df_unmatched['lowertag'].str.strip()


with open("gerwel.py", "r") as file:
    for line in file:
        gerwel.append(line.strip())

print(gerwel)
gerwel_set = set(gerwel)

#Deze zijn volgens Ger WEL een hoedanigheid
df_ger_unmatched = df_unmatched[df_unmatched['lowertag'].isin(gerwel_set)]
len(df_ger_unmatched)

In [ ]:
df_unmatched.to_csv('df_ger_unmatched.tsv', sep='\t')
df_unmatched_q = df_unmatched[~df_unmatched['lowertag'].isin(gerwel_set)] 
df_unmatched_q.to_csv('ger_checken.tsv', sep='\t')#Dit zijn de twijfelgevallen
len(df_unmatched_q) 

In [ ]:
# Als Ger heeft gezegd dat het geen hoedanigheid is, drop it. 
gerniet = []
df_unmatched['lowertag'] = df_unmatched['lowertag'].str.strip()

with open("gerniet.py", "r") as file:
    for line in file:
        gerniet.append(line.strip())

print(gerniet)
gerniet_set = set(gerniet)

df_unmatched['delete'] = df_unmatched.apply(lambda row: 'x' if row['lowertag'] in gerniet_set else row['delete'], axis=1)
df.update(df_unmatched)
df_delete = df[df['delete'] == 'x']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]
df_unmatched = df_unmatched.drop(common_indices)
len(df_unmatched)


lijst ter organisatie

In [ ]:
import pandas as pd

# Group df_matched by 'anno_name1' (Hoedanigheid) and find the top three 'tag' values with the highest corresponding 'aantal'
top_tags = df_matched.groupby('anno_name1').apply(lambda x: x.nlargest(3, 'aantal')).reset_index(drop=True)

# Merge the top three 'tag' values back into df_grouped based on the 'Hoedanigheid' column
enriched_df_grouped = pd.merge(df_grouped, top_tags.groupby('anno_name1')['tag'].apply(lambda x: ';'.join([f'"{tag}"' for tag in x])).reset_index(), how='left', left_on='Hoedanigheid', right_on='anno_name1')

# Rename the merged column to 'tags' and drop the redundant 'anno_name1' column
enriched_df_grouped = enriched_df_grouped.rename(columns={'tag': 'tags'}).drop(columns='anno_name1')

# Print the enriched DataFrame
enriched_df_grouped.head(100)

In [ ]:
# Rename the "Pattern" column to "Patterns"
enriched_df_grouped = enriched_df_grouped.rename(columns={'Pattern': 'Patterns'})
enriched_df_grouped = enriched_df_grouped.rename(columns={'Categorie': 'Categorie GD'})
enriched_df_grouped = enriched_df_grouped.rename(columns={'Dictionary': 'Categorie NG'})

# Drop the first column
enriched_df_grouped = enriched_df_grouped.drop(columns=enriched_df_grouped.columns[0])

# Reorganize columns
enriched_df_grouped = enriched_df_grouped[['Hoedanigheid', 'Categorie GD', 'Level_2', 'Categorie NG', 'Patterns', 'tags']]

# Print the updated DataFrame
print(enriched_df_grouped)


In [ ]:
enriched_df_grouped['Categorie GD'] = enriched_df_grouped['Categorie GD'].apply(lambda x: ', '.join(sorted(set(str(x).split(', ')) - {''})))
enriched_df_grouped.to_csv('HOED_cats.tsv', sep='\t')

In [ ]:
hoedanigheden = new_df['anno_name1'].unique().tolist()
len(hoedanigheden)

In [ ]:
import pickle

# Assuming hoedanigheden is your list

# Define the file name for the pickle file
pickle_file = "hoedanigheden.pkl"

# Dump the list into the pickle file
with open(pickle_file, "wb") as f:
    pickle.dump(hoedanigheden, f)

print(f"List 'hoedanigheden' has been dumped into '{pickle_file}' successfully.")
